In [5]:
import numpy as np
import pandas as pd

In [6]:
merged = pd.read_csv('../data/ranked_athletes.csv')
merged = merged.drop(columns='birth_year')
merged[:50]

,Unnamed: 0,ID,Rank,Last Name,First Name,Points,Event,Gender,Year,country,height,weight,age
0,0,8372,1,Ondra,Adam,300.0,lead,MEN,2019,CZE,185.0,67.0,26.0
1,1,8372,2,Ondra,Adam,335.0,boulder,MEN,2019,CZE,185.0,67.0,26.0
2,2,8372,31,Ondra,Adam,55.0,lead,MEN,2018,CZE,185.0,67.0,25.0
3,3,8372,28,Ondra,Adam,80.0,lead,MEN,2017,CZE,185.0,67.0,24.0
4,4,8372,24,Ondra,Adam,51.0,lead,MEN,2016,CZE,185.0,67.0,23.0
5,5,8372,1,Ondra,Adam,458.0,lead,MEN,2015,CZE,185.0,67.0,22.0
6,6,8372,3,Ondra,Adam,259.0,boulder,MEN,2015,CZE,185.0,67.0,22.0
7,7,8372,3,Ondra,Adam,428.0,lead,MEN,2014,CZE,185.0,67.0,21.0
8,8,8372,24,Ondra,Adam,80.0,boulder,MEN,2014,CZE,185.0,67.0,21.0
9,9,8372,11,Ondra,Adam,180.0,lead,MEN,2013,CZE,185.0,67.0,20.0


In [22]:
men_lead_count_dfs = {}
num_events_per_year = {}
discipline = 'bouldering'
for year in range(2019, 1990, -1):
    filepath = '../data/worldcup_data.csv'
    df = pd.read_csv(filepath, index_col=0)
    df = df.drop(columns=['rank', 'title', 'date', 'comp_id', 'cat_id'])
    year_df = df[(df['year']==year) & (df['type']==discipline)]
    counts = year_df['athlete_id'].value_counts()
    men_lead_count_dfs[year] = counts
    num_events = max(counts) if len(counts) else 0
    num_events_per_year[year] = num_events

In [24]:
top_athletes = merged['ID'].unique()
drop_points = False

# ml suffix is for 'men' 'lead'
merged_ml = merged[(merged['Event']=='boulder') & (merged['Gender']=='MEN')].copy()
merged_ml = merged_ml.drop(columns=['Event', 'Gender'])
merged_ml = merged_ml.reset_index(drop=True)

#Adding the career length feature
athlete_ids = merged_ml['ID'].unique()
experience = pd.DataFrame()
experience_list = []
for ID in athlete_ids:
    athlete = merged_ml[merged_ml['ID']==ID]
    starting_year = athlete['Year'].min()
    athlete_experience = athlete['Year'] - starting_year
    experience_list.append(athlete_experience)
merged_ml['career_len'] = pd.concat(experience_list)


# Add lag columns
all_counts = []
for i, row in merged_ml.iterrows():
    year = row['Year']
    ID = row['ID']
    if year in men_lead_count_dfs:
        count_df = men_lead_count_dfs[year]
        if ID in count_df:
            all_counts.append(count_df[ID])
        else:
            merged_ml.drop(i)
    else:
        merged_ml.drop(i)
    
merged_ml['event_count'] = all_counts

# TO DO: We can implement the thing where an athletes lowest score is dropped
if drop_points:
    raise NotImplementedError()
#     num_events = num_events_per_year[]
#     merged_ml['avg_points'] = None
else:
    merged_ml['avg_points'] = merged_ml['Points'] / merged_ml['event_count']

K = 7 # num of lags
col_names = [f't-{k+1}' for k in range(K)]
lag_df = pd.DataFrame(columns=col_names)
for i, ID in enumerate(top_athletes):
    athlete = merged_ml[merged_ml['ID']==ID].copy()
    lag_list = [athlete['avg_points'].shift(-(k+1)) for k in range(K)]
    lags = pd.concat(lag_list, axis=1)
    lags.columns = col_names
    lag_df = pd.concat([lag_df, lags], axis=0)

merged_ml = pd.concat([merged_ml, lag_df], axis=1)
merged_ml = merged_ml.rename(columns={'ID': 'id', 
                                      'Rank': 'rank',
                                      'Last Name': 'last_name',
                                      'First Name': 'first_name',
                                      'Points': 'points',
                                      'Year': 'year'})
merged_ml.head(30) # This one goes by athlete, rather than by year

ValueError: Length of values (1514) does not match length of index (1589)

In [12]:
sorted_merged_ml = merged_ml.sort_values(by=['year', 'points'], ascending=False)
sorted_merged_ml = sorted_merged_ml.reset_index(drop=True)
sorted_merged_ml.to_csv('../data/men_boulder_no_drop.csv', index=False)
sorted_merged_ml

KeyError: 'year'